In [2]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/README.md
/kaggle/input/dataset/IIVP_PROJ.ipynb
/kaggle/input/dataset/test/test/Fear/1.jpg
/kaggle/input/dataset/test/test/Fear/3.jpg
/kaggle/input/dataset/test/test/Fear/2.jpg
/kaggle/input/dataset/test/test/Angry/6.jpg
/kaggle/input/dataset/test/test/Angry/5.jpg
/kaggle/input/dataset/test/test/Angry/8.jpg
/kaggle/input/dataset/test/test/Angry/10.jpg
/kaggle/input/dataset/test/test/Angry/9.jpg
/kaggle/input/dataset/test/test/Angry/1.jpg
/kaggle/input/dataset/test/test/Angry/7.jpg
/kaggle/input/dataset/test/test/Angry/11.jpg
/kaggle/input/dataset/test/test/Angry/4.jpg
/kaggle/input/dataset/test/test/Angry/3.jpg
/kaggle/input/dataset/test/test/Angry/2.jpg
/kaggle/input/dataset/test/test/Neutral/45.jpg
/kaggle/input/dataset/test/test/Neutral/20.jpg
/kaggle/input/dataset/test/test/Neutral/6.jpg
/kaggle/input/dataset/test/test/Neutral/5.jpg
/kaggle/input/dataset/test/test/Neutral/8.jpg
/kaggle/input/dataset/test/test/Neutral/30.jpg
/kaggle/input/dataset/test/test/Neut

In [8]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = datasets.ImageFolder(root = '/kaggle/input/dataset/train/train',
                                             #train = True,
                                             transform = all_transforms,
                                             )

# Create Testing dataset
test_dataset = datasets.ImageFolder(root = '/kaggle/input/dataset/test/test',
                                            #train = False,
                                            transform = all_transforms,
                                            )

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [16]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [17]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

In [19]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        if torch.cuda.is_available():
        # Move tensors to the configured device
            device= torch.device('cuda')
            images = images.to(device)
            labels = labels.to(device)
            model = model.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

            
            


Epoch [1/20], Loss: 2.1625
Epoch [2/20], Loss: 1.6179
Epoch [3/20], Loss: 1.6396
Epoch [4/20], Loss: 1.1250
Epoch [5/20], Loss: 1.1622
Epoch [6/20], Loss: 1.1914
Epoch [7/20], Loss: 1.1393
Epoch [8/20], Loss: 1.1282
Epoch [9/20], Loss: 1.0489
Epoch [10/20], Loss: 1.2421
Epoch [11/20], Loss: 0.9911
Epoch [12/20], Loss: 1.1262
Epoch [13/20], Loss: 1.0625
Epoch [14/20], Loss: 1.1638
Epoch [15/20], Loss: 1.3655
Epoch [16/20], Loss: 1.0873
Epoch [17/20], Loss: 1.3175
Epoch [18/20], Loss: 0.9385
Epoch [19/20], Loss: 0.9601
Epoch [20/20], Loss: 1.3038


In [24]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))




Accuracy of the network on the 50000 train images: 59.436619718309856 %
